In [1]:
%cd ..
%cd ..

/home/ltorres/leo/tesis/cloud-classification/code/notebooks
/home/ltorres/leo/tesis/cloud-classification/code


In [2]:
%load_ext autoreload
%autoreload 2

---

In [3]:
from torchvision.io import read_image
import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from random import sample
import torch
import torch.nn as nn
from torch.optim import lr_scheduler

In [4]:
from src.dataset import GCD, GRSCD
from src import config
from src import engine_gnn as engine
from src import utils

from src.models.graph_nets import GraphConvGNN, GATConvGNN

from notebooks.adjacency_construction.adjacency_models import *

In [5]:
from sklearn.metrics import accuracy_score

In [6]:
import wandb

#### PARAMS

In [6]:
device = 'cuda:1'


BUILDER = 'mlp'
builder_name = "mlp"

LR = 3e-5
EPOCHS = 1000



exp_name = '2_GATConvGNN_MLP_LOGE_SGD_GRSCD_06_05_22'
model_filename = f'/grscd_gnn/{exp_name}.pt'

---

In [8]:
wandb.login()

wandb: Currently logged in as: leonardtd (use `wandb login --relogin` to force relogin)


True

#### Data loaders

In [9]:
path_train_images = utils.get_grscd_paths(config.DATA_DIR,'train')

In [10]:
train_dataset = GRSCD(path_train_images, resize=256, aug_types='augmentation')

train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config.BATCH_SIZE,
        num_workers=4,
        shuffle=True,
    )

In [20]:
path_test_images = utils.get_grscd_paths(config.DATA_DIR,'test')

test_dataset = GRSCD(path_test_images, resize=256)

test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=config.BATCH_SIZE,
        num_workers=6,
        shuffle=True,
    )

### Data augmentation data loaders

In [12]:
aug_dataset = GRSCD(path_train_images, resize=256, aug_types='augmentation')

augmentation_loader = torch.utils.data.DataLoader(
        aug_dataset,
        batch_size=config.BATCH_SIZE,
        num_workers=4,
        shuffle=True,
    )

In [13]:
print(len(train_dataset))
print(len(test_dataset))
print(len(aug_dataset))

4000
4000
4000


#### Model

In [14]:
from torchvision import models

In [15]:
torch.cuda.empty_cache()

In [8]:
model = GATConvGNN(7, 1024, 2, 4, 0.7, device, adjacency_builder = BUILDER).to(device)

In [9]:
#criterion = utils.loge_loss
criterion = nn.CrossEntropyLoss()
optimizer  = torch.optim.SGD(model.parameters(), lr=LR, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

####  wandb

In [18]:
wandb.init(
    # Set the project where this run will be logged
    project="grscd",
    # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    name=exp_name, 
    # Track hyperparameters and run metadata
    config={
    "learning_rate": LR,
    "architecture": "GATConvGNN",
    "loss": "LogeLoss",
    "optim": "SGD",
    "dataset": "GCD",
    "builder": f"{builder_name}",
    "epochs": EPOCHS,
  })

### Train

In [ ]:
best_accuracy=0
best_iteration=0
best_params={}

for e in range(EPOCHS):
    ### TRAIN DATASET
    preds, targets, loss = engine.train_fn(model, train_loader, criterion, optimizer, device=device)
    
    # ### AUGMENTATION IMAGES
    # engine.forward_backward_pass(model, augmentation_loader, criterion, optimizer, device=device)
    
    #scheduler.step()
    if e%20==0:
        train_acc = accuracy_score(targets, preds)
        
        test_preds, test_targets, test_loss = engine.eval_fn(model, test_loader, criterion, device=device)
        test_acc = accuracy_score(test_targets, test_preds)

        if test_acc > best_accuracy:
            torch.save(model.state_dict(), config.SAVE_PATH+model_filename)
            print(f"Saved best parameters at epoch {e+1}")
            best_accuracy = test_acc
            best_iteration = e+1
            best_params = model.state_dict()
    
        print("EPOCH {}: Train acc: {:.2%} Train Loss: {:.4f} Test acc: {:.2%} Test Loss: {:.4f}".format(
            e+1,
            train_acc,
            loss,
            test_acc,
            test_loss
        ))

        metrics = {
                    "train/train_loss": loss,
                    "train/train_accuracy": train_acc,
                    "test/test_loss": test_loss,
                    "test/test_accuracy": test_acc,
                  }

        wandb.log(metrics)


100%|██████████| 125/125 [00:28<00:00,  4.42it/s]


Saved best parameters at epoch 1
EPOCH 1: Train acc: 22.48% Train Loss: 3.7825 Test acc: 48.33% Test Loss: 3.5008


100%|██████████| 125/125 [00:37<00:00,  3.32it/s]


Saved best parameters at epoch 21
EPOCH 21: Train acc: 95.45% Train Loss: 0.3522 Test acc: 81.47% Test Loss: 1.0074


100%|██████████| 125/125 [00:37<00:00,  3.33it/s]


...

100%|██████████| 125/125 [00:37<00:00,  3.38it/s]


EPOCH 721: Train acc: 99.95% Train Loss: 0.0037 Test acc: 81.95% Test Loss: 1.9866


100%|██████████| 125/125 [00:37<00:00,  3.34it/s]


EPOCH 741: Train acc: 100.00% Train Loss: 0.0025 Test acc: 83.58% Test Loss: 1.7055


 96%|█████████▌| 120/125 [00:41<00:01,  3.51it/s]

In [ ]:
wandb.finish()

In [18]:
model.load_state_dict(torch.load(config.SAVE_PATH+model_filename))

<All keys matched successfully>

In [ ]:
test_preds, test_targets, test_loss = engine.eval_fn(model, test_loader, criterion, device='cpu')

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
accuracy_score(test_targets, test_preds)

---

### Accuracy per class and confusion matrix

In [ ]:
class_mapper = {
    0: '1_cumulus',
    1: '2_altocumulus',
    2: '3_cirrus',
    3: '4_clearsky',
    4: '5_stratocumulus',
    5: '6_cumulonimbus',
    6: '7_mixed',
}

In [ ]:
class_acc = dict()
matrix = np.zeros((7,7), dtype=int)

for i in range(7):
    pred_index = np.where(test_targets==i)[0]
    class_acc[class_mapper[i]] = accuracy_score(test_targets[pred_index], test_preds[pred_index])
    
    for j in test_preds[pred_index]:
        matrix[i,j]+=1 

In [ ]:
class_acc

In [ ]:
print(f"Global Test accuracy {accuracy_score(test_targets, test_preds)}")

sum_of_rows = matrix.sum(axis=1)
conf_mat = 100*matrix / sum_of_rows[:, np.newaxis]

df_cm = pd.DataFrame(conf_mat, index = class_acc.keys(),
                  columns = class_acc.keys())

plt.figure(figsize = (8,5))
plt.title("Confusion Matrix (Accuracy %)")
sns.heatmap(df_cm, annot=True)